# Endimensjonal kvantemekanisk brønnmodell for atomer, molekyler og krystaller

I denne notatboken skal vi undersøke en enkel kvantemekanisk modell for atomer, molekyler og krystaller der potensialet omkring enkeltatomer modelleres med brønnpotensialer.

Omkring en atomkjerne vil det være et potensiale som elektroner med masse $m_e$ kan befinne seg i.
I vår enkle, endimensjonale modell vil vi beskrive dette potensialet som et brønnpotensial

$$V(x) = \begin{cases} -V_0 & |x| \leq w / 2 \\ 0 & \text{ellers} \\ \end{cases}$$

omkring et atom med bredde $w$ sentrert i origo.
For å beskrive molekyler og atomer, konstruerer vi nye potensialer som består av like mange slike brønnpotensialer som det er atomer i molekylet eller krystallen, plassert etter hverandre og separert med områder med null potensiale og bredde $b$ som modellerer bindinger og bindingslengder i strukturen.

Vi neglisjerer vekselvirkning mellom elektroner rundt atomkjernen, slik at hvert elektron befinner seg i en stasjonær tilstand $\Psi(x, t) = \psi(x) e^{i E t / \hbar}$ med tilhørende energi $E$, der energiegenfunksjonen $\psi(x)$ løser den tidsuavhengige Schrödingerligningen

$$-\frac{\hbar^2}{2 m_e} \psi''(x) + V(x) \psi(x) = E \psi(x)$$

der $V(x)$ er et (annet) potensial og $\hbar$ er Plancks konstant.

Elektronet er et fermion som kan ha to spinnverdier, og Paulis eksklusjonsprinsipp forteller oss da at maksimalt to elektroner vil okkupere hver stasjonære tilstand.

Schrödingerligningen har sjelden analytiske løsninger.
Derfor vil vi her løse ligningen numerisk ved å hakke opp rommet i $N + 2$ punkter $x_0 = 0, x_1, \ldots, x_N, x_{N+1} = L$ med avstand $\Delta x = L / (N + 1)$ mellom hvert punkt.
Hvert punkt tilordnes potensialet $V_n = V(x_n)$ og verdien $\psi_n = \psi(x_n)$ for energiegenfunksjonen.
For å avgrense problemet, innfører vi randbetingelsene $V_0 = V_{N+1} = \infty$, slik at $\psi_0 = \psi_{N+1} = 0$.
I tillegg tilnærmer vi de deriverte av energiegenfunksjonen med
$$\psi_n' = \psi'(x_n) = \frac{\psi_{n+1} - \psi_n}{\Delta x} \qquad \text{og} \qquad \psi_n'' = \psi''(x_n) = \frac{\psi_n' - \psi_{n-1}'}{\Delta x} = \frac{\psi_{n+1} - 2 \psi_n + \psi_{n-1}}{{\Delta x}^2}$$

Problemet er dermed redusert til å finne verdier $\psi_1, \ldots, \psi_N$ for energiegenfunksjonen og tilhørende energier $E$ som oppfyller
$$-\frac{\hbar^2}{2 m_e} \frac{\psi_{n+1} - 2 \psi_n + \psi_{n-1}}{{\Delta x}^2} + V_n \psi_n = E \psi_n$$
Dette er ekvivalent med å finne egenvektorer $\boldsymbol{\psi} = [\psi_1, \ldots, \psi_N]^T$ og tilhørende egenverdier $E$ som oppfyller matriseligningen
$$H \boldsymbol{\psi} = E \boldsymbol{\psi}$$
der matrisen $H$ er en symmetrisk matrise med $H_{n, n} = \hbar^2 / m {\Delta x}^2 + V_n$ langs diagonalen, $H_{n, n \pm 1} = \hbar^2 / 2 m {\Delta x}^2$ over og under diagonalen og $0$ alle andre steder.
Dette er et velkjent problem fra lineær algebra med mange effektive programvareimplementasjoner, og er utgangspunktet for vår numeriske løsningsmetode.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

WAVE_SCALE_FACTOR = 100 # Factor to scale wave with, so that when plotting we get
                        # y = engy + WAVE_SCALE_FACTOR * wave
hbar = 1.05e-25 # nJs
m = 9.11e-31 # kg
eV = 1.60e-19 # J

try:
    from numpy import heaviside
    heaviside = heaviside
except:
    heaviside = lambda x,a: (x > 0) * 1

# Some does not have scipy.linalg.eigh_tridiagonal, so this takes care of that
# TODO: leave only one in the end
try:
    from scipy.linalg import eigh_tridiagonal
    get_eigh = lambda H: eigh_tridiagonal(np.diag(H), np.diag(H, k=1))
except ImportError:
    get_eigh = np.linalg.eigh
    
def V_crystal(x, w, b, L, V0):
    n = (x - 10 * w) / (w + b)
    V = np.where((x > 10 * w) & (x < L - 10 * w + b) & ((n - np.floor(n)) * (w + b) < w), -V0, 0)
    return V

def partition_space(N, N_w, w, b):
    L = 10 * w + N_w * w + (N_w - 1) * b + 10 * w
    dx = L / (N + 1)
    x = np.linspace(dx, N * dx, N)
    return L, dx, x

def get_stationary_states(dx, V):
    H = np.zeros((N, N))
    for i in range(0, N):
        H[i][i] = hbar**2 / (m * dx**2) / eV + V[i]
    for i in range(0, N - 1):
        H[i][i+1] = -hbar**2 / (2 * m * dx**2) / eV
        H[i+1][i] = -hbar**2 / (2 * m * dx**2) / eV
    
    eigvals, eigvecs = get_eigh(H)
    energies, waves = eigvals, eigvecs.T
    return energies, waves

def plot_potential(x, V, **kwargs):
    plt.plot(x, V, color=kwargs.pop("potential_color", "grey"),
                linestyle=kwargs.pop("potential_style", "solid"),
                linewidth=kwargs.pop("potential_width", 3),
                label="$V(x)$")

def plot_stationary_states(x, energies, waves, L,
                           include_engy_legend = True, **kwargs):
    
    for n, (energy, wave) in enumerate(zip(energies, waves)):
        color = "C%d" % (n % 10)
        label = "$E_{%d} = %+.1f \,\, eV, \psi_{%d}$" % (n, energy, n) if include_engy_legend else None
        plt.plot((0, L), (energy, energy), color=color, linestyle="dashed", label=label)
        
        plt.plot(x, energy + wave * WAVE_SCALE_FACTOR, color=color, linestyle="solid")
    
def investigate_potential(N, N_w, w, b, n, V0,
                          show_legend = True,
                          clip_y_axis = False,
                          **kwargs):
    """
    :include_engy_legend : whether to include labels for energy in legend
    :clip_y_axis         : sets upper limit of y_axis to biggest wave value,
                           so that if potential is much bigger than wave, we can "zoom-in"
    """
    L, dx, x = partition_space(N, N_w, w, b)
    V = V_crystal(x, w, b, L, V0)
    
    energies, waves = get_stationary_states(dx, V)
    
    print("Greatest deviation from orthonormality:")
    print(np.max(np.abs(waves @ waves.T - np.eye(N))))

    plt.figure(figsize=(16, 8))
    plt.title("%d lowest-energy states with %d wells, well width %.2f and well distance %.2f" % (n, N_w, w, b))
    plt.xlabel("$x \,\, (nm)$")
    plt.ylabel("$V(x), E_n \,\, (eV)$")
    plot_potential(x, V, **kwargs)
    plot_stationary_states(x, energies[:n], waves[:n], L, **kwargs) # Pass **kwargs to gain better control over plotting
    
    if clip_y_axis:
        # Sets upper y limit to biggest wave
        y_max = np.amax( WAVE_SCALE_FACTOR * waves[:n] + energies[:n,np.newaxis] )    
        axes = plt.gca()
        axes.set_ylim([None, y_max + 0.1 * np.abs(y_max)])
    
    plt.legend(loc="upper left")
    plt.show()

N = 400
w = 0.1 # nm
b = w / 5 # nm
n = 6
V0 = 200


investigate_potential(N, 0, w, b, n, V0)
investigate_potential(N, 1, w, b, n, V0)
investigate_potential(N, 2, w, b, n, V0)
investigate_potential(N, 10, w, b, n, V0)

# Eksempel på bruk av plottekontroller
num_wells = 10
num_engy_to_show = 4

investigate_potential(N, num_wells, w, b, num_engy_to_show, V0,
                      include_engy_legend = False,
                      clip_y_axis = True,
                      potential_style = "dashed",
                      potential_width = 2)

# Oppgave 2

### a)
Figuren viser en potensialbrønn med fire tilhørende bølgefunksjoner. Brønnens dybde er her valgt slik at det finnes tre bølgefunksjoner innenfor brønnen, mens dybden er satt til $0.1 \, nm $ -- omtrent tilsvarende et atoms radius. Dette for å oppnå tall av en mer naturtro størrelsesorden. 

Fra figuren kommer det frem at annenhver bølgeunksjon er symmetrisk og antisymmetrisk, hvilket stemmer med teori. Man ser også at bølgefunksjonene for grunntilstand, første og andre eksiterte tilstand henholdsvis har 0, 1 og 2 nullpunkter. De oppfyller også initialbetingelsene om at de skal være kontinuerlige og at de går mot null i stor avstand fra brønnen, når potensialet er null.

Denne potensialfunksjonen kan modellere et atom, og bølgefunksjonene er da mulige tilstander elektroner kan befinne seg i. 
### TODO: Sjekk energiegenverdiene opp mot analytiske verdier.

Det er tegnet inn en bølgefunksjon ovenfor brønnen, i tillegg til de tre i brønnen. Denne representerer bølgefunksjonen for et ubundet elektron (fritt elektron), som har for stor kinetisk energi til å fanges av det elektriske feltet til atomkjernen. Et slikt elektron skal, langt unna atomet, ha en upåvirket, sinusoidal bølgefunksjon. Bølgelengden til elektronet leses fra grafen av til å være omkring $2 \, nm$.
### TODO Les av bølgelengden. Si litt mer om elektronet?

### b)
Med tre bundne tilstander er det plass til inntil seks elektroner innenfor potensialbrønnen. Dette fordi Pauliprinsippet utelukker muligheten for at det kan eksistere mer enn to elektroner (ett med spin opp og ett med spinn ned) i hver bundne tilstand. En slik potensialbrønn kan modellere atomer av nummer én til og med seks (hydrogen til og med karbon), som har ett til seks elektroner i grunntilstand. Disse atomenes totale elektronspinn finnes fra Pauliprinsippet. Dersom et atom har et partall antall elektroner (slik som i helium og karbon), vil dets totale elektronpinn i grunntilstand være null. Dersom det har et odde antall elektroner (f.eks. hydrogen, litium), blir spinnet $S = \pm \frac{\sqrt{3}}{2} \hbar$. I eksiterte tilstander kan spinnet også ta andre verdier, alle heltallige multiplum av $\pm \frac{\sqrt{3}}{2} \hbar$.

# Oppgave 3

Dobbel potensialbrønn, hver med samme dybde og lengde som i oppgave 2.
Kan modellere et molekyl.
Får dobbelt så mange bundne stasjonære tilstander som i oppgave 2, dvs. totalt 6.

Ser av figuren at to separate H-atomer til sammen har energien $2 \cdot (-179.0 \, eV) = -358.0 \, eV$ i grunntilstanden.
Et H2-molekyl har energien $2 \cdot (-179.6 \, eV) = -359.2 \, eV$ i grunntilstanden.
H2-molekylet har dermed en bindingsenergi $(-359.2 \, eV) - (-358.0 \, eV) = -1.2 \, eV$ relativt de separate H-atomene.
Den negative differansen indikerer at H2-molekylet er mer stabilt enn de separate H-atomene, noe som stemmer med erfaring.

To separate He-atomer har til sammen energien $2 \cdot 2 \cdot (-179 \, eV) = -716.0 \, eV$.
Et He2-molekyl har energien $2 \cdot (-179.6 \, eV) + 2 \cdot (-175.5 \, eV) = -710.2 \, eV$.
He2-molekylet har dermed en bindingsenergi $(-710.2 \, eV) - (-716.0 \, eV) = + 5.8 \, eV$ relativt de separate He-atomene.
Den positive differansen indikerer at He2-molekylet er mindre stabilt enn de separate He-atomene, noe som stemmer med erfaring, da He er en edelgass.

Disse resultatene kan knyttes opp mot teori om bindende og antibindende orbitaler.
I H2-molekylet okkuperer begge elektronene en energitilstand med noe lavere energi enn grunntilstanden for et H-atom.
En slik energitilstand gjør molekylet mer stabilt enn de separate atomene, og kalles derfor en bindende orbital.
I He2-molekylet okkuperer to av elektronene en energitilstand med noe lavere energi enn grunntilstanden for et He-atom, og to av elektronene en energitilstand med noe høyere energi enn grunntilstanden for et He-atom.
Til sammen får molekylet en høyere energi enn de separate atomene og er derfor minst stabilt, og den høyeste energitilstanden av de to i molekylet kalles derfor en antibindende orbital, på grunn av denne effekten.

<!-- TODO: sammenlign kvantitativt (i hvert fall relativt mellom H og He) fra tall i Wikipedia-artikler -->
<!-- https://en.wikipedia.org/wiki/Helium_dimer -->
<!-- https://en.wikipedia.org/wiki/Bond-dissociation_energy -->
<!-- https://en.wikipedia.org/wiki/Antibonding_molecular_orbital -->

# Om nøyaktigheten til modellen

Det er viktig å understreke at brønnmodellen er en primitiv modell for atomer, molekyler og krystaller.
Den vil ikke gi nøyaktige eller kvantitativt korrekte tallverdier, men den lar oss belyse enkelte kvalitative egenskaper.
For eksempel vil vi her benytte en brønn med en fast verdi for brønndybden $V_0$ til å modellere ulike atomer med ulike antall protoner i kjernen, selv om brønndybden rundt ulike atomer med forskjellige antall protoner i kjernen i realiteten vil variere.
For å ha et rimelig bilde av situasjonene vi modellerer, vil vi regne med korrekte verdier for Plancks (reduserte) konstant og elektronmassen og en rimelig verdi for brønnbredden, som i denne modellen tilsvarer atombredden på omtrent 0.1 nm.
For å tvinge frem observerbare forskjeller på de ulike situasjonene og illustrere kvalitative forskjeller, er vi derimot nødt til å redusere bindingslengden mellom atomer noe under dens realistiske verdi, og til slutt brønndybden til å gi et ønsket antall bundne tilstander.